# Functional Responses

**How well do different mathematical models, e.g., based upon foraging theory (mechanistic) principles vs. phenomenological ones, fit to functional responses data across species?**

Functional responses:
  * Are the relationships between a consumer's biomass consumption rate and abundance of a target resource. 

  * Arise from fundamental biological and physical constraints on consumer-resource interactions.

  * Determine the rate of biomass flow between species in ecosystems across the full scale of sizes. 

  * Play a key role in determining the stability of food webs that underpin ecosystems

NB: Two main fields of interest are **N_TraitValue** (the number of resources consumed per consumer per unit time) and **ResDensity** (the resource abundance). 

Each ID value corresponds to one curve and one individual functional response curve

In [165]:
# Some imports to explore the datasets
import pandas as pd
import scipy as sc
import matplotlib.pylab as pl
import seaborn as sns
import numpy as np
import csv

## Getting Started

In [144]:
data = pd.read_csv("../data/CRat.csv")
print("Loaded {} columns.".format(len(data.columns.values)))

Loaded 68 columns.


In [145]:
print(data.columns.values)

['ID' 'DataType' 'ORIGINAL_TraitID' 'ORIGINAL_TraitDefinition' 'TraitValue'
 'TraitUnit' 'N_TraitValue' 'ConTaxa' 'ResTaxa' 'ConTaxaStage'
 'ResTaxaStage' 'Con_ForagingMovement' 'Con_RESDetectionDimensionality'
 'Res_ForagingMovement' 'Res_CONDetectionDimensionality' 'CON_MASS_value'
 'RES_MASS_value' 'ResArenaSize_SI_UNIT' 'ResDensity_SI_VALUE'
 'ResDensityUnit' 'ResDensity' 'ResArenaSize_SI_VALUE' 'ConTemp' 'ResTemp'
 'ResReplaceRate' 'ResReplaceUnit' 'ResReplace' 'TraitSIValue'
 'TraitSIUnit' 'N_TraitID' 'N_TraitConversion' 'N_CONVERTED' 'N_TraitUnit'
 'Original_ErrorValue' 'Original_ErrorValueUnit' 'Replicates' 'ConTaxon'
 'ConStage' 'ConCommon' 'Con_MovementDimensionality' 'Con_Thermy'
 'Res_MovementDimensionality' 'ResTaxon' 'ResStage' 'ResCommon'
 'Res_Thermy' 'Habitat' 'LabField' 'ObservationtimeSI'
 'ObservationtimeSIUnits' 'ConStarvationTimeSI' 'ConStarvationTimeSIUnits'
 'ConArenaSiz_VALUE' 'ConArenaSize_UNIT' 'ConDensity' 'ConDensityUnit'
 'ConDensity_SI_VALUE' 'ConDensityC

#### Making new data frame with only columns of interest for initial plots

In [146]:
newdata = data[['ID','ResDensity','N_TraitValue']].copy()
print(newdata)

         ID  ResDensity  N_TraitValue
0     39835     10.0000      0.001206
1     39835     10.0000      0.001065
2     39835     10.0000      0.001206
3     39835     25.0000      0.002562
4     39835     25.0000      0.002411
5     39835     25.0000      0.002009
6     39835     25.0000      0.001587
7     39835     50.0000      0.004330
8     39836     19.9323      0.000525
9     39836     40.1737      0.000950
10    39836     60.0883      0.001321
11    39836     80.3182      0.001647
12    39836    100.2190      0.001900
13    39836    120.1230      0.002181
14    39836    140.3380      0.002371
15    39836    160.2320      0.002570
16    39836    180.4470      0.002760
17    39836    200.3350      0.002904
18    39837     19.9323      0.000255
19    39837     40.1737      0.000483
20    39837     60.0883      0.000702
21    39837     80.3182      0.000894
22    39837    100.2190      0.001076
23    39837    120.1230      0.001241
24    39837    140.3380      0.001387
25    39837 

Check if there are any columns with NA values

In [147]:
newdata2 = newdata[newdata.isna().any(axis=1)]
print(newdata2)

Empty DataFrame
Columns: [ID, ResDensity, N_TraitValue]
Index: []


#### Removing IDs with less than 5 data points

In [148]:
newdata.groupby('ID').ID.count() # grouping data points by ID number and storing a count of instances

ID
2          4
3          4
6         32
71         6
79        10
140        6
351       21
445        4
450        6
687       60
691       60
695        5
708        5
713       31
721        6
814        9
920        6
39835      8
39836     10
39837     10
39838     10
39839     10
39840     36
39841     41
39842     34
39843     28
39844     42
39845     24
39846     23
39847     31
        ... 
40100     40
40101     72
40102    101
40103      8
40104      8
40105      6
40106      5
40107      5
40108     63
40109     70
40110     72
40111      6
40112      5
40114      5
40115      6
40116      6
40117      6
40118      6
40119      6
40120      6
40121      6
40122     30
40123      6
40124      6
40125     21
40126     21
40127     16
40128      7
40129      8
40130      5
Name: ID, Length: 308, dtype: int64

Setting a threshold number of repeats. If a ID has less than the threshold number of repeats, all of its data is replaced with null values

In [149]:
threshold = 5
ValueCounts = newdata['ID'].value_counts()
toremove = ValueCounts[ValueCounts <= threshold].index
newdata.replace(toremove, np.nan, inplace=True)

In [150]:
newdata['ID'].value_counts() #Counting number of repeated IDs 

40050.0    142
40051.0    105
40102.0    101
40043.0     88
40042.0     88
39940.0     80
40047.0     73
40110.0     72
40101.0     72
40048.0     71
40052.0     70
40109.0     70
40053.0     70
40054.0     70
40055.0     70
40108.0     63
691.0       60
687.0       60
39883.0     50
40049.0     48
39911.0     42
39844.0     42
40033.0     42
39841.0     41
39910.0     40
40099.0     40
40100.0     40
39925.0     36
39840.0     36
40039.0     34
          ... 
40069.0      6
40070.0      6
40071.0      6
40072.0      6
40073.0      6
40074.0      6
40075.0      6
40076.0      6
40077.0      6
40078.0      6
40080.0      6
40081.0      6
40082.0      6
40084.0      6
40111.0      6
40085.0      6
40086.0      6
40087.0      6
40088.0      6
40089.0      6
40090.0      6
40091.0      6
40092.0      6
40093.0      6
40094.0      6
40095.0      6
40096.0      6
40097.0      6
40105.0      6
39936.0      6
Name: ID, Length: 241, dtype: int64

Remove all rows with NA data

In [153]:
newdata_no_missing = newdata.dropna()

In [154]:
newdata_no_missing.isnull().sum() #count number of NAs in data set to be sure

ID              0
ResDensity      0
N_TraitValue    0
dtype: int64

#### Save modified data into a .csv file

In [164]:
newdata_no_missing.to_csv('../data/FunResData.csv')

## NLLS Fitting Script